In [1]:
# Instalação das bibliotecas necessárias
# Execute apenas uma vez
!pip install tensorflow opencv-python matplotlib numpy

   ---------------------------------------- 0.0/332.0 MB ? eta -:--:--
   ---------------------------------------- 1.6/332.0 MB 10.0 MB/s eta 0:00:34
   - -------------------------------------- 9.7/332.0 MB 26.7 MB/s eta 0:00:13
   -- ------------------------------------- 24.1/332.0 MB 42.2 MB/s eta 0:00:08
   ---- ----------------------------------- 41.4/332.0 MB 53.3 MB/s eta 0:00:06
   ------ --------------------------------- 53.5/332.0 MB 54.1 MB/s eta 0:00:06
   ------ --------------------------------- 57.4/332.0 MB 55.4 MB/s eta 0:00:05
   -------- ------------------------------- 74.2/332.0 MB 52.0 MB/s eta 0:00:05
   ---------- ----------------------------- 85.2/332.0 MB 52.0 MB/s eta 0:00:05
   ----------- --------------------------- 100.7/332.0 MB 54.4 MB/s eta 0:00:05
   ------------- ------------------------- 117.4/332.0 MB 57.0 MB/s eta 0:00:04
   --------------- ----------------------- 129.0/332.0 MB 57.6 MB/s eta 0:00:04
   --------------- ----------------------- 129.0/33


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#Imports
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import tensorflow as tf
from pathlib import Path
import random

d:\programming-stuffs\projects\project-naive-bayes-web\website\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


# Aumento de Dataset (Data Augmentation)

Este notebook aplica técnicas de aumento de dados nas imagens originais para expandir o dataset:

**Técnicas aplicadas:**
- Imagens Originais
- Flip Horizontal (espelhamento)
- Ajuste de Tom (Hue)
- Flip + Ajuste de Tom

**Resultado:** Cada imagem original gera 4 variações, multiplicando o tamanho do dataset por 4x.

**Saída:** As imagens processadas serão salvas em `naive_bayes_training/dataset_finalized/`

In [3]:

# Configurações de Caminho
path_raiz = Path("dataset")
directory_path = path_raiz

def salvar_imagem(img_data, pasta, nome):
    if hasattr(img_data, 'numpy'):
        dados = img_data.numpy()
    else:
        dados = img_data
    
    # NOVO: Garante que os dados estão no range [0, 1] e são float32
    dados = np.clip(dados, 0, 1)
    
    path_dest = pasta / nome
    # NOVO: cmap=None garante que ele não tente aplicar mapas de cores estranhos
    plt.imsave(str(path_dest), dados)

# Mapeamento de nomes das pastas originais para as finalizadas
nome_mapeamento = {
    "dataset_severe_accident": "dataset_final_severe_accident",
    "dataset_moderate_accident": "dataset_final_moderate_accident",
    "dataset_no_accident": "dataset_final_no_accident"
}

for folder_path in directory_path.iterdir():
    if folder_path.is_dir() and folder_path.name in nome_mapeamento:
        
        print(f"\n--- Processando Categoria: {folder_path.name} ---")

        # Usa o nome mapeado para a pasta final
        nome_final = nome_mapeamento[folder_path.name]
        new_name_folder = path_raiz.parent / "naive_bayes_training" / "dataset_finalized" / nome_final
        new_name_folder.mkdir(parents=True, exist_ok=True)

        for path_arquivo in folder_path.iterdir():
            if path_arquivo.suffix.lower() in ['.png', '.jpg', '.jpeg']:
                try:
                    # NOVO: Leitura e Normalização Imediata
                    img_raw = mpimg.imread(str(path_arquivo))
                    
                    # Se a imagem for 0-255 (inteiro), divide por 255 para virar 0-1 (float)
                    if img_raw.dtype == np.uint8 or np.max(img_raw) > 1.0:
                        img_raw = img_raw.astype(np.float32) / 255.0
                    
                    # NOVO: Remover canal Alpha (transparência) se existir
                    if img_raw.shape[-1] == 4:
                        img_raw = img_raw[:, :, :3]

                    # Converter para Tensor explicitamente
                    img_original = tf.cast(img_raw, tf.float32)

                    # --- 1. Original ---
                    salvar_imagem(img_original, new_name_folder, f"Original_{path_arquivo.name}")

                    # --- 2. Flip ---
                    img_flip = tf.image.flip_left_right(img_original)
                    salvar_imagem(img_flip, new_name_folder, f"Flip_{path_arquivo.name}")

                    # --- 3. Tom (Hue) ---
                    # Hue delta deve ser pequeno (entre -0.5 e 0.5)
                    img_tom = tf.image.adjust_hue(img_original, delta=random.uniform(-0.1, 0.1))
                    salvar_imagem(img_tom, new_name_folder, f"Tom_{path_arquivo.name}")

                    # --- 4. Flip + Tom (Hue) ---
                    # Delta positivo clareia, negativo escurece. Use valores pequenos.
                    img_claridade = tf.image.adjust_hue(img_flip, delta=random.uniform(-0.1, 0.1))
                    salvar_imagem(img_claridade, new_name_folder, f"Flip_Tom_{path_arquivo.name}")

                    print(f"Sucesso: {path_arquivo.name}")

                except Exception as e:
                    print(f"Erro em {path_arquivo.name}: {e}")

print("\n🚀 Processamento concluído! Verifique as pastas em 'naive_bayes_training/dataset_finalized/'.")


--- Processando Categoria: dataset_moderate_accident ---
Sucesso: 000001.jpg
Sucesso: 000002.jpg
Sucesso: 000003.jpg
Sucesso: 000004.jpg
Sucesso: 000005.jpg
Sucesso: 000006.jpg
Sucesso: 000007.jpg
Sucesso: 000008.jpg
Sucesso: 000009.jpg
Sucesso: 000010.jpg
Sucesso: 000011.jpg
Sucesso: 000012.jpg
Sucesso: 000013.jpg
Sucesso: 000014.jpg
Sucesso: 000015.jpg
Sucesso: 000016.jpg
Sucesso: 000017.jpg
Sucesso: 000018.jpg
Sucesso: 000019.jpg
Sucesso: 000020.jpg
Sucesso: 000021.jpg
Sucesso: 000022.jpg
Sucesso: 000023.jpg
Sucesso: 000024.jpg
Sucesso: 000025.jpg
Sucesso: 000026.jpg
Sucesso: 000027.jpg
Sucesso: 000028.jpg
Sucesso: 000029.jpg
Sucesso: 000030.jpg
Sucesso: 000031.jpg
Sucesso: 000032.jpg
Sucesso: 000033.jpg
Sucesso: 000034.jpg
Sucesso: 000035.jpg
Sucesso: 000036.jpg
Sucesso: 000037.jpg
Sucesso: 000038.jpg
Sucesso: 000039.jpg
Sucesso: 000040.jpg
Sucesso: 000041.jpg
Sucesso: 000042.jpg
Sucesso: 000043.jpg
Sucesso: 000044.jpg
Sucesso: 000045.jpg
Sucesso: 000046.jpg
Sucesso: 000047.jpg
Su